In [1]:
import pandas as pd
import numpy as np
from espn_api.football import League
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

True

In [2]:
# Access the variables
league_id = os.getenv("league_id")
espn_s2 = os.getenv("espn_s2")
swid = os.getenv("swid")

In [3]:
league = League(league_id, 2025, espn_s2, swid, debug=False)

In [5]:
league.previousSeasons

[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

In [15]:
# Define the column names
columns = [
    "year",
    "week",
    "home_team",
    #"home_owner",
    "home_score",
    "home_result",
    "away_team",
    #"away_owner",
    "away_score",
    "away_result"
]

# Create an empty DataFrame
league_history = pd.DataFrame(columns=columns)

In [16]:
def get_weekly_matchups(lg, wk):
    """ Returns all matchups from a given week. """

    return lg.scoreboard(wk)

In [17]:
def create_matchup_dict(yr, wk, matchup):
    """ Creates a dictionary for a given matchup """

    matchup_dict = {
                'year': yr,
                'week': wk,
                'home_team': matchup.home_team.team_name,
                #'home_owner': matchup.home_team.owners[0]['firstName'],
                'home_score': matchup.home_score,
                'home_result': 0,
                'away_team': matchup.away_team.team_name,
                #'away_owner': matchup.away_team.owners[0]['firstName'],
                'away_score': matchup.away_score,
                'away_result': 0
    }

    if matchup_dict['home_score'] > matchup_dict['away_score']:
        matchup_dict['home_result'] = 1
    elif matchup_dict['home_score'] < matchup_dict['away_score']:
        matchup_dict['away_result'] = 1
    else:
        matchup_dict['home_result'] = 0.5
        matchup_dict['away_result'] = 0.5

    return matchup_dict

In [18]:
def append_matchup_to_league_history(lg_hist, matchup_dict):
    """ Appends a provided matchup to the league's history """

    lg_hist = pd.concat([lg_hist, matchup_dict], ignore_index = True)

    return lg_hist

In [19]:
def create_season_history(yr, lg, lg_hist):
    """ Appends all matchups from a season into the league_history_data frame. """

    current_season_weeks = [i + 1 for i in range(lg.current_week - 1)]

    for i in current_season_weeks:
        wk_scb = get_weekly_matchups(lg, i)

        num_matchups = len(wk_scb)

        for j in range(len(wk_scb)):
            game_result = create_matchup_dict(2025, i, wk_scb[j])

            lg_hist = pd.concat([lg_hist, pd.DataFrame([game_result])], ignore_index = True)

    return lg_hist

In [20]:
create_season_history(2025, league, league_history)

C:\Users\Paul\AppData\Local\Temp\ipykernel_11252\3613636817.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  lg_hist = pd.concat([lg_hist, pd.DataFrame([game_result])], ignore_index = True)


,year,week,home_team,home_score,home_result,away_team,away_score,away_result
0,2025,1,Baker mayflower,65.28,0,Big Boppers,94.78,1
1,2025,1,Sun God Hostage Holders,110.42,1,The Molders,93.42,0
2,2025,1,SkatteDOOM,103.02,0,No Hats,106.42,1
3,2025,1,Space miners,80.36,0,Florida Man,101.42,1
4,2025,1,RIP Joe's Toes,83.12,0,Bebe Bullets,86.12,1
...,...,...,...,...,...,...,...,...
61,2025,11,The Molders,79.10,1,Space miners,72.42,0
62,2025,11,No Hats,60.54,0,RIP Joe's Toes,90.20,1
63,2025,11,Florida Man,116.22,0,Dragon Deez Nuts,119.98,1
64,2025,11,Bebe Bullets,119.56,1,Baker mayflower,115.32,0
